In [8]:
import requests as rqt

In [9]:
standings_url="https://fbref.com/en/comps/9/Premier-League-Stats"

In [10]:
data=rqt.get(standings_url)
data

<Response [429]>

In [11]:
from bs4 import BeautifulSoup

In [12]:
soup = BeautifulSoup(data.text)

In [13]:
standings_table=soup.select('table.stats_table')[0]
standings_table

IndexError: list index out of range

In [ ]:
links=standings_table.find_all('a')

In [ ]:
links=[l.get('href') for l in links]

In [ ]:
team_links=[l for l in links if '/squads' in l]

In [ ]:
team_urls=[f"https://fbref.com{l}" for l in team_links]

In [ ]:
team_urls=team_urls[0]

In [ ]:
data=rqt.get(team_urls)
data

<Response [200]>

In [ ]:
import pandas as pd

matches=pd.read_html(data.text,match="Scores & Fixtures")

NameError: name 'data' is not defined

In [ ]:
soup=BeautifulSoup(data.text)

In [ ]:
links=soup.find_all('a')

In [ ]:
links=[l.get('href') for l in links]

In [ ]:
links=[l for l in links if l and 'all_comps/shooting/' in l]
links

['/en/squads/b8fd03ef/2023-2024/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2023-2024/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2023-2024/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2023-2024/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions']

In [ ]:
data=rqt.get(f"https://fbref.com{links[0]}")
data

<Response [200]>

In [ ]:
shooting=pd.read_html(data.text,match="Shooting")[0]

In [ ]:
shooting.columns=shooting.columns.droplevel()

In [ ]:
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (1),1 (4),Arsenal,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,13.9,0.0,0,0,1.9,1.9,0.12,1.1,1.1,Match Report
2,2023-08-16,22:00,Super Cup,UEFA Super Cup,Wed,Home,D,1 (5),1 (4),es Sevilla,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,17.9,0.0,0,0,1.0,1.0,0.07,0.0,0.0,Match Report
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,17.3,2.0,0,1,3.5,2.8,0.10,-1.5,-0.8,Match Report


In [ ]:
team_data=matches[0].merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]],on="Date")

In [ ]:
shooting.shape

(58, 26)

In [ ]:
years=list(range(2022,2020,-1))

In [ ]:
standings_url="https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
all_matches=[]

In [ ]:
import time
for year in years:
  data=rqt.get(standings_url)
  print(data)
  soup = BeautifulSoup(data.text)
  standings_table=soup.select('table.stats_table')[0]
  links=standings_table.find_all('a')
  links=[l.get('href') for l in links]
  team_links=[l for l in links if '/squads' in l]
  team_urls=[f"https://fbref.com{l}" for l in team_links]

  previous_season=soup.select("a.prev")[0].get('href')
  standings_url=f"https://fbref.com{previous_season}"

  for team_url in team_urls:
    team_name=team_url.split("/")[-1].replace("-Stats","").replace("-"," ")
    data=rqt.get(team_url)
    matches=pd.read_html(data.text,match="Scores & Fixtures")
    soup=BeautifulSoup(data.text)
    links=soup.find_all('a')
    links=[l.get('href') for l in links]
    links=[l for l in links if l and 'all_comps/shooting/' in l]
    data=rqt.get(f"https://fbref.com{links[0]}")
    shooting=pd.read_html(data.text,match="Shooting")[0]
    shooting.columns=shooting.columns.droplevel()

    try:
      team_data=matches[0].merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]],on="Date")
    except ValueError:
      continue
    team_data=team_data[team_data["Comp"]=="Premier League"]
    team_data["Season"]=year
    team_data["Team"]=team_name
    all_matches.append(team_data)
    time.sleep(2)


NameError: name 'rqt' is not defined

In [ ]:
match_df=pd.concat(all_matches)

In [ ]:
match_df.to_csv('matches.csv')